In [7]:
import torchvision
from keras.models import load_model
import numpy as np
import torch
import torch.nn as nn
import time

## Transformer le modèle keras h5 en pytorch

In [4]:
#Recreer un modele pytorch pour stocker notre modèle keras dedans

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)  

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, h):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, h)
        self.fc2 = nn.Linear(h, h)
        self.fc3 = nn.Linear(h, h)
        self.fc4 = nn.Linear(h, h)
        self.fc5 = nn.Linear(h, action_size)
        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(*hidden_init(self.fc3))
        self.fc4.weight.data.uniform_(*hidden_init(self.fc4))
        self.fc5.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

        return F.tanh(self.fc5(x))

net=Actor(1,1,time.time(),32)

In [5]:
#On charge le modele
model = load_model('tomnod_everything_relu_Adam.h5')

#On recupere ses poids
weights=model.get_weights()

#On transpose les poids dans le modèle pytorch
net.fc1.weight.data=torch.from_numpy(np.transpose(weights[0]))
net.fc1.bias.data=torch.from_numpy(weights[1])
net.fc2.weight.data=torch.from_numpy(np.transpose(weights[2]))
net.fc2.bias.data=torch.from_numpy(weights[3])
net.fc3.weight.data=torch.from_numpy(np.transpose(weights[4]))
net.fc3.bias.data=torch.from_numpy(weights[5])
net.fc4.weight.data=torch.from_numpy(np.transpose(weights[6]))
net.fc4.bias.data=torch.from_numpy(weights[7])
net.fc5.weight.data=torch.from_numpy(np.transpose(weights[8]))
net.fc5.bias.data=torch.from_numpy(weights[9])

In [21]:
#Faux data set pour tester
entries=10000
x=np.random.randn(entries,1)*np.pi
y=np.sin(x)+(np.random.randn(entries,1)*.1)  
xReal=np.linspace(-np.pi*3,np.pi*3,entries/100)
xReal=np.transpose(np.array([xReal]))
yReal=np.sin(xReal) 

pyPredict0=net.forward(torch.from_numpy(xReal).float())

TypeError: 'float' object cannot be interpreted as an integer

## Torchvision

In [ ]:
# Pour geler toutes les couches sauf la dernière 

for param in model.parameters():  
    param.requires_grad = False

In [41]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [42]:
#Definir la fonction de transformation
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Définir le jeu de données d'entraînement (par exemple CIFAR10)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform) 

# Définir le DataLoader pour les données d'entraînement
train_loader = DataLoader(train_dataset, batch_size=64,
                          shuffle=True, num_workers=0)

Files already downloaded and verified


In [45]:
# Charger un modèle pré-entraîné (ici ResNet18)
model = torchvision.models.resnet18(pretrained=True)

# Définir la dernière couche en fonction du nombre de classes dans votre jeu de données
num_classes = 10 # nombre de classes
model.fc = nn.Linear(512, num_classes)

# Définir les paramètres à optimiser
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        
# Configurer l'optimiseur (SGD avec une faible vitesse d'apprentissage)
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# Définir la fonction de perte (cross-entropy)
criterion = nn.CrossEntropyLoss()

#Définir les épochs 
num_epochs = 50 


In [ ]:
# Boucle d'entraînement
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / (i + 1)))

print('Finished Training')